In [ ]:
import tensorflow as tf
tf.random.set_seed(1234)
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import pickle
import math

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
train_labels1 = tf.keras.utils.to_categorical(train_labels)
test_labels1 = tf.keras.utils.to_categorical(test_labels)

In [ ]:
BATCH_SIZE=64
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels1))
train_ds = train_ds.shuffle(train_images.shape[0]).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels1))
test_ds = train_ds.shuffle(test_images.shape[0]).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
class CustomMLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.multiply_no_nan(x=y_true,
                                                   y=tf.math.divide_no_nan(x=tf.math.subtract(x=y_true, y=y_pred), y=y_pred))
        #element_wise = tf.math.add(x=tf.math.divide_no_nan(x=tf.math.subtract(x=y_true, y=y_pred), y=y_pred),
        #                   y=tf.math.divide_no_nan(x=tf.math.subtract(x=1.0, y=y_true), y=tf.math.subtract(x=1.0, y=y_pred)))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CustomLLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.divide_no_nan(x=y_true,
                                             y=tf.math.sqrt(
                                                 x=tf.math.subtract(x=1.0,
                                                                    y=tf.math.squared_difference(x=y_pred, y=1.0))
                                             ))

        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CustomLFullLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.subtract(x=tf.math.add(x=tf.math.divide_no_nan(x=y_true,
                                             y=tf.math.sqrt(
                                                 x=tf.math.subtract(x=1.0,
                                                                    y=tf.math.squared_difference(x=y_pred, y=1.0))
                                             )),
                                                      y=tf.math.divide_no_nan(x=tf.math.subtract(x=1.0,y=y_true),
                                                                              y=tf.math.sqrt(
                                                                                  tf.math.subtract(x=1.0,y=tf.math.square(y_pred))
                                                                              ))),
                                        y=1.0)

        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class My_CE_Loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        log_y_pred = tf.math.log(y_pred)
        element_wise = -tf.math.multiply_no_nan(x=log_y_pred, y=y_true)
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))


In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import SGD, Adam

base_model = VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(32, 32, 3),
    classes=10
)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='tanh')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

opt = Adam(learning_rate=1e-5)
model.compile(optimizer=opt, loss=CustomMLoss(), metrics=['accuracy'])

In [ ]:
def create_model():
    inputs = tf.keras.layers.Input(shape=(32,32,3))
    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)

    base_model_output = tf.keras.applications.vgg16.VGG16(
        include_top=False,
        input_shape=(224, 224, 3),
        weights='imagenet'
    )(resize)

    x = tf.keras.layers.GlobalAveragePooling2D()(base_model_output)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    predictions = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
    return model

In [ ]:
model = create_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=CustomMLoss(),
    metrics=['accuracy'],
)
history = model.fit(train_ds, epochs=50, verbose=2)